In [ ]:
import json
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import string
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from string import punctuation
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
with open('/content/drive/MyDrive/capstone_data/train.json') as file:
    train_data = json.load(file)

with open('/content/drive/MyDrive/capstone_data/validation.json') as file:
    validation_data = json.load(file)

In [ ]:
df = pd.read_csv('/content/kesehatan.csv')
df.head()

,Judul Berita,Link
0,Ferry Irawan Divonis 1 Tahun Penjara di Kasus ...,https://www.cnnindonesia.com/nasional/20230523...
1,PKS Akui Bukhori Yusuf Anggota DPR yang Diduga...,https://www.cnnindonesia.com/nasional/20230523...
2,Bareskrim Dalami Kasus Dugaan KDRT Anggota DPR...,https://www.cnnindonesia.com/nasional/20230523...
3,Pengacara Politikus PKS Terjerat KDRT Sebut Pe...,https://www.cnnindonesia.com/nasional/20230523...
4,"Dialami Megan Fox, Apa Itu Body Dysmorphobia?",https://www.cnnindonesia.com/gaya-hidup/202305...


In [ ]:
df['label'] = 'kesehatan'
data = df.copy()

In [ ]:
df.head()

,Judul Berita,Link,label
0,Ferry Irawan Divonis 1 Tahun Penjara di Kasus ...,https://www.cnnindonesia.com/nasional/20230523...,kesehatan
1,PKS Akui Bukhori Yusuf Anggota DPR yang Diduga...,https://www.cnnindonesia.com/nasional/20230523...,kesehatan
2,Bareskrim Dalami Kasus Dugaan KDRT Anggota DPR...,https://www.cnnindonesia.com/nasional/20230523...,kesehatan
3,Pengacara Politikus PKS Terjerat KDRT Sebut Pe...,https://www.cnnindonesia.com/nasional/20230523...,kesehatan
4,"Dialami Megan Fox, Apa Itu Body Dysmorphobia?",https://www.cnnindonesia.com/gaya-hidup/202305...,kesehatan


In [ ]:
data.head()

,Judul Berita,Link,label
0,Ferry Irawan Divonis 1 Tahun Penjara di Kasus ...,https://www.cnnindonesia.com/nasional/20230523...,kesehatan
1,PKS Akui Bukhori Yusuf Anggota DPR yang Diduga...,https://www.cnnindonesia.com/nasional/20230523...,kesehatan
2,Bareskrim Dalami Kasus Dugaan KDRT Anggota DPR...,https://www.cnnindonesia.com/nasional/20230523...,kesehatan
3,Pengacara Politikus PKS Terjerat KDRT Sebut Pe...,https://www.cnnindonesia.com/nasional/20230523...,kesehatan
4,"Dialami Megan Fox, Apa Itu Body Dysmorphobia?",https://www.cnnindonesia.com/gaya-hidup/202305...,kesehatan


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Judul Berita  50 non-null     object
 1   Link          50 non-null     object
 2   label         50 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [ ]:
stopwords.words('indonesian')

['ada',
 'adalah',
 'adanya',
 'adapun',
 'agak',
 'agaknya',
 'agar',
 'akan',
 'akankah',
 'akhir',
 'akhiri',
 'akhirnya',
 'aku',
 'akulah',
 'amat',
 'amatlah',
 'anda',
 'andalah',
 'antar',
 'antara',
 'antaranya',
 'apa',
 'apaan',
 'apabila',
 'apakah',
 'apalagi',
 'apatah',
 'artinya',
 'asal',
 'asalkan',
 'atas',
 'atau',
 'ataukah',
 'ataupun',
 'awal',
 'awalnya',
 'bagai',
 'bagaikan',
 'bagaimana',
 'bagaimanakah',
 'bagaimanapun',
 'bagi',
 'bagian',
 'bahkan',
 'bahwa',
 'bahwasanya',
 'baik',
 'bakal',
 'bakalan',
 'balik',
 'banyak',
 'bapak',
 'baru',
 'bawah',
 'beberapa',
 'begini',
 'beginian',
 'beginikah',
 'beginilah',
 'begitu',
 'begitukah',
 'begitulah',
 'begitupun',
 'bekerja',
 'belakang',
 'belakangan',
 'belum',
 'belumlah',
 'benar',
 'benarkah',
 'benarlah',
 'berada',
 'berakhir',
 'berakhirlah',
 'berakhirnya',
 'berapa',
 'berapakah',
 'berapalah',
 'berapapun',
 'berarti',
 'berawal',
 'berbagai',
 'berdatangan',
 'beri',
 'berikan',
 'berikut'

In [ ]:
data['Judul Berita'] = data['Judul Berita'].str.lower()

In [ ]:
data.head()

,Judul Berita,Link,label
0,ferry irawan divonis 1 tahun penjara di kasus ...,https://www.cnnindonesia.com/nasional/20230523...,kesehatan
1,pks akui bukhori yusuf anggota dpr yang diduga...,https://www.cnnindonesia.com/nasional/20230523...,kesehatan
2,bareskrim dalami kasus dugaan kdrt anggota dpr...,https://www.cnnindonesia.com/nasional/20230523...,kesehatan
3,pengacara politikus pks terjerat kdrt sebut pe...,https://www.cnnindonesia.com/nasional/20230523...,kesehatan
4,"dialami megan fox, apa itu body dysmorphobia?",https://www.cnnindonesia.com/gaya-hidup/202305...,kesehatan


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Judul Berita  50 non-null     object
 1   Link          50 non-null     object
 2   label         50 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [ ]:
# def remove_punctuation(text):
#     trans_table = str.maketrans('', '', punctuation)
#     return text.translate(trans_table)

# text = remove_punctuation(data['Judul Berita'])

# tokens = word_tokenize(text)

# filtered_tokens = ' '.join([token for token in tokens if token.lower() not in stopwords.words('indonesian')])
# filtered_tokens

In [ ]:
def labelling(json_data):
    keywords, theme, recom = [], [], []
    theme_label = {}

    for news in json_data['berita']:
        for keyword in news['keywords']:
            keywords.append(keyword)
            theme.append(news['tema_berita'])
            recom.append(news['recommendation'])

        for tema in news['tema_berita']:
            if tema not in theme_label:
                theme_label[news['tema_berita']] = news['recommendation']

    return keywords, theme, recom, theme_label

In [ ]:
train_keywords, train_theme, train_recom, train_theme_label = labelling(train_data)

val_keywords, val_theme, val_recom, val_theme_label = labelling(validation_data)

In [ ]:
data = pd.DataFrame({
    'keywords': train_keywords,
    'theme': train_theme,
    'recommendation': train_recom
})

data.sample(10)

,keywords,theme,recommendation
14,evakuasi,kebakaran,"[Kebakaran, Pemadam Kebakaran]"
20,percikan,kebakaran,"[Kebakaran, Pemadam Kebakaran]"
95,investigasi,polisi,"[Kejahatan, Polisi]"
63,jasad,polisi,"[Kejahatan, Polisi]"
119,sebaran,rumah sakit,"[Rumah Sakit, Penyakit, Kesehatan]"
133,pengobatan,rumah sakit,"[Rumah Sakit, Penyakit, Kesehatan]"
153,mengonsumsi,rumah sakit,"[Rumah Sakit, Penyakit, Kesehatan]"
138,vaksin,rumah sakit,"[Rumah Sakit, Penyakit, Kesehatan]"
113,dokter,rumah sakit,"[Rumah Sakit, Penyakit, Kesehatan]"
102,tanggung jawab,polisi,"[Kejahatan, Polisi]"


In [ ]:
tokenizer = Tokenizer()
def preprocess_text(train_data, val_data):
    all_data = train_data + val_data
    tokenizer.fit_on_texts(all_data)
    train_sequences = tokenizer.texts_to_sequences(train_data)
    val_sequences = tokenizer.texts_to_sequences(val_data)
    word_index = tokenizer.word_index
    max_sequence_length = max(len(seq) for seq in train_sequences + val_sequences)
    padded_train_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length)
    padded_val_sequences = pad_sequences(val_sequences, maxlen=max_sequence_length)
    return padded_train_sequences, padded_val_sequences, word_index, max_sequence_length

label_mapping = {'kebakaran': 0, 'polisi': 1, 'rumah sakit': 2}
num_classes = len(label_mapping)

X_train, X_test, word_index, max_sequence_length = preprocess_text(train_keywords, val_keywords)
# X_train, _, word_index = preprocess_text(train_keywords, val_keywords)
y_train = to_categorical([label_mapping[label] for label in train_theme[:len(X_train)]], num_classes=num_classes)

# _, X_test, _ = preprocess_text(train_keywords, val_keywords)
y_test = to_categorical([label_mapping[label] for label in val_theme[:len(X_test)]], num_classes=num_classes)

In [ ]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, 100))
model.add(LSTM(128))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/100
11/11 [==============================] - 4s 81ms/step - loss: 1.0996 - accuracy: 0.3210 - val_loss: 1.0962 - val_accuracy: 0.3452
Epoch 2/100
11/11 [==============================] - 0s 15ms/step - loss: 1.0932 - accuracy: 0.3333 - val_loss: 1.0920 - val_accuracy: 0.4167
Epoch 3/100
11/11 [==============================] - 0s 15ms/step - loss: 1.0804 - accuracy: 0.5000 - val_loss: 1.0851 - val_accuracy: 0.5536
Epoch 4/100
11/11 [==============================] - 0s 17ms/step - loss: 1.0631 - accuracy: 0.6049 - val_loss: 1.0733 - val_accuracy: 0.5595
Epoch 5/100
11/11 [==============================] - 0s 18ms/step - loss: 1.0248 - accuracy: 0.7963 - val_loss: 1.0504 - val_accuracy: 0.5893
Epoch 6/100
11/11 [==============================] - 0s 16ms/step - loss: 0.9579 - accuracy: 0.9568 - val_loss: 1.0134 - val_accuracy: 0.5536
Epoch 7/100
11/11 [==============================] - 0s 23ms/step - loss: 0.8407 - accuracy: 0.9630 - val_loss: 0.9458 - val_accuracy: 0.6131
Epoch 

In [ ]:
def preprocess_input(title):
    tokens = title.apply(word_tokenize)
    tokens = [token for sublist in tokens for token in sublist if token not in punctuation]
    text = ' '.join([token for token in tokens if token not in stopwords.words('indonesian')])
    return text

test_text = preprocess_input(df['Judul Berita'])
test_text

'Ferry Irawan Divonis 1 Tahun Penjara Kasus KDRT Venna Melinda PKS Akui Bukhori Yusuf Anggota DPR Diduga Lakukan KDRT Bareskrim Dalami Kasus Dugaan KDRT Anggota DPR Fraksi PKS Pengacara Politikus PKS Terjerat KDRT Sebut Pelapor Pernah Depresi Dialami Megan Fox Apa Itu Body Dysmorphobia Viral Kebanyakan Makan Jengkol Disebut Bikin Gagal Ginjal Flu Burung Merebak Brasil Berlakukan Darurat Kesehatan Hewan 180 Hari 5 Tanda Bahaya Otak Menua Lebih Cepat Usiamu Apakah Pengidap Kanker Payudara Tetap Bisa Menyusui PM Singapura Positif Covid-19 Kunjungan Afrika Selatan Bocah SD Sukabumi Tewas Diduga Dianiaya Teman Sekolah Erick Thohir Kirim Dokter Rumah Kurnia Meiga Ahli Rekomendasikan Skrining Kanker Payudara Usia 40 Tahun Doa Minum Air Zam-Zam Arab Latin Artinya Arti Syafakillah Syafakallah beserta Cara Menjawabnya Kecemasan Akan Krisis Iklim Disebut Bisa Berpengaruh Pada Seks Happy Bee-day 6 Fakta Unik Soal Lebah Tak Banyak Diketahui Studi Ungkap Cuaca Panas Terus-menerus Bisa Bikin Gangguan

In [ ]:
test_sentence = """
Kebakaran Pabrik Plastik Jakbar Tak Ganggu Penerbangan Bandara Soetta
"""

test_sequence = tokenizer.texts_to_sequences([test_sentence])
padded_test_sequence = pad_sequences(test_sequence, maxlen=max_sequence_length)
prediction = model.predict(padded_test_sequence)[0]

recommended_hobby_index = tf.argmax(prediction).numpy()
recommended_hobby = [label for label, index in label_mapping.items() if index == recommended_hobby_index][0]

print("Rekomendasi: " + recommended_hobby)

1/1 [==============================] - 0s 34ms/step
Rekomendasi: kebakaran
